In [1]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from gurobipy import GRB, Model

In [2]:
static = pd.read_csv('static_with_t.csv')
detected = pd.read_csv('randomized_graph.csv')

In [3]:
node_name = 'conn_f_30'

In [4]:
static_node = static[static['name'] == node_name].to_dict('records')[0]
static_node

{'name': 'conn_f_30',
 'number': 30,
 'x': 12.0,
 'y': -86.0,
 'z': 0.0,
 'parent': 'cross_f30',
 'distance_to_parent': 39}

In [5]:
detected_node = detected[detected['name'] == node_name].to_dict('records')[0]
detected_node

{'name': 'conn_f_30',
 'number': 30,
 'x': 11.28,
 'y': -86.0,
 'z': -4.1,
 'parent': 'cross_f30'}

In [6]:
detected_parent = detected[detected['name'] == static_node['parent']].to_dict('records')[0]
detected_parent


{'name': 'cross_f30',
 'number': -1,
 'x': -2.82,
 'y': -50.0,
 'z': 1.03,
 'parent': 'cross_f'}

In [7]:
parent_x = detected_parent["x"]
parent_y = detected_parent["y"]
parent_z = detected_parent["z"]

detected_x = detected_node["x"]
detected_y = detected_node["y"]
detected_z = detected_node["z"]

distance_to_parent = static_node["distance_to_parent"]

In [8]:
quadratic_model = Model('quadratic')

Restricted license - for non-production use only - expires 2024-10-28


In [9]:
x = quadratic_model.addVar(name = 'x', vtype=GRB.CONTINUOUS)
y = quadratic_model.addVar(name = 'y', vtype=GRB.CONTINUOUS)
z = quadratic_model.addVar(name = 'z', vtype=GRB.CONTINUOUS)

In [10]:
detected_x_rel = detected_x - parent_x
detected_y_rel = detected_y - parent_y
detected_z_rel = detected_z - parent_z

print(f"detected_x: {detected_x}, detected_y: {detected_y}, detected_z: {detected_z}")
print(f"parent_x: {parent_x}, parent_y: {parent_y}, parent_z: {parent_z}")
print(f"detected_x_rel: {detected_x_rel}, detected_y_rel: {detected_y_rel}, detected_z_rel: {detected_z_rel}")

detected_x: 11.28, detected_y: -86.0, detected_z: -4.1
parent_x: -2.82, parent_y: -50.0, parent_z: 1.03
detected_x_rel: 14.1, detected_y_rel: -36.0, detected_z_rel: -5.13


In [11]:
quadratic_model.addQConstr(
    x**2 + y**2 + z**2 <= distance_to_parent**2)

<gurobi.QConstr Not Yet Added>

In [12]:
distance_to_parent

39

In [13]:
detected_x_rel

14.1

In [14]:
detected_y_rel

-36.0

In [15]:
detected_z_rel

-5.13

In [16]:
obj_fn = (x - detected_x_rel)**2 + (y - detected_y_rel)**2 + (z - detected_z_rel)**2

In [17]:
quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)

In [18]:
quadratic_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x7f43b456
Model has 3 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+01, 7e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+03, 2e+03]
Presolve time: 0.02s
Presolved: 8 rows, 9 columns, 12 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.400e+01
 Factor NZ  : 3.600e+01
 Factor Ops : 2.040e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       P

In [19]:
calc_x = quadratic_model.getVarByName('x').x
calc_y = quadratic_model.getVarByName('y').x
calc_z = quadratic_model.getVarByName('z').x

In [20]:
#add rel
calc_x_f = calc_x + parent_x
calc_y_f = calc_y + parent_y
calc_z_f = calc_z + parent_z

In [21]:
print(f"calc_x: {calc_x}, calc_y: {calc_y}, calc_z: {calc_z}")
print(f"detected_x_rel: {detected_x_rel}, detected_y_rel: {detected_y_rel}, detected_z_rel: {detected_z_rel}")
print(f"calc_x_f: {calc_x_f}, calc_y_f: {calc_y_f}, calc_z_f: {calc_z_f}")
print(f"detected_x: {detected_x}, detected_y: {detected_y}, detected_z: {detected_z}")
print(f"parent_x: {parent_x}, parent_y: {parent_y}, parent_z: {parent_z}")

calc_x: 14.09941009378733, calc_y: 1.5891274714609427e-10, calc_z: 9.117020055937355e-10
detected_x_rel: 14.1, detected_y_rel: -36.0, detected_z_rel: -5.13
calc_x_f: 11.27941009378733, calc_y_f: -49.99999999984109, calc_z_f: 1.030000000911702
detected_x: 11.28, detected_y: -86.0, detected_z: -4.1
parent_x: -2.82, parent_y: -50.0, parent_z: 1.03


In [22]:
print('Obj: %g' % np.sqrt(quadratic_model.objVal))

Obj: 36.3637


In [23]:
print(f"function value: {np.sqrt(quadratic_model.objVal)}")
print(f"Distance to detected node: {np.sqrt((calc_x-detected_x_rel)**2 + (calc_y-detected_y_rel)**2 + (calc_z-detected_z_rel)**2)}")
print(f"calculated distance to parent: {np.sqrt((calc_x_f-parent_x)**2 + (calc_y_f-parent_y)**2 + (calc_z_f-parent_z)**2)}")
print(f"Original distance to parent: {distance_to_parent}")


function value: 36.36367560821755
Distance to detected node: 36.36367556186785
calculated distance to parent: 14.09941009378733
Original distance to parent: 39


In [24]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from transformation_utils import get_rotation, rotate_graph, get_rotation_nn, optim_rotation

G = nx.Graph()

G.add_node('static_node', pos=(static_node['x'], static_node['z'], static_node['y']))
G.add_node('detected_node', pos=(detected_node['x'], detected_node['z'], detected_node['y']))
G.add_node('detected_parent', pos=(detected_parent['x'], detected_parent['z'], detected_parent['y']))
G.add_node('calculated_node', pos=(calc_x_f, calc_z_f, calc_y_f))

plot_graph(static_graph=G, detected_graph=None, name='qcqp', save=True)




Graph saved to qcqp.html
